In [2]:
!pip install transformers torch wandb evaluate huggingface_hub datasets evaluate numpy peft accelerate sacrebleu

In [3]:
import numpy as np
import pandas as pd
import torch
import os
from torch.utils.data import DataLoader, random_split
from datasets import load_dataset, Dataset

# USE RAY TUNE. https://docs.ray.io/en/latest/train/examples/intel_gaudi/bert.html
# deepl, chatgpt translations

## Setup

In [4]:
import wandb
from huggingface_hub import HfApi, HfFolder
import transformers

try: # If it is on Kaggle
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()

    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    WANDB_KEY = user_secrets.get_secret("WANDB_KEY")

except ModuleNotFoundError: # If it is local
    HF_TOKEN = os.environ["HF_TOKEN"]
    WANDB_KEY = os.environ["WANDB_KEY"]
    

HfFolder.save_token(HF_TOKEN)
wandb.login(key=WANDB_KEY)


# Reproducibility

seed = 1
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
transformers.set_seed(seed)
np.random.seed(seed)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abdulmohsena to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
2025-06-23 23:01:10.865285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750719671.033226      49 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750719671.082633      49 cuda_blas.cc:1418] U

## Modeling

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig
from transformers import DataCollatorForSeq2Seq

In [6]:
# # # # Configure any model from HF HUB
# assert input("YOU WILL REMOVE THE HUB MODEL FOR THIS, TYPE 'OK' TO PROCEED: ").upper() == 'OK'
# model_name = "facebook/mbart-large-50-many-to-many-mmt"
# model_name = "facebook/m2m100_1.2B"
# #model_name= "Helsinki-NLP/opus-mt-en-ar"
# model_name= "facebook/nllb-200-distilled-600M"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# generation_config = GenerationConfig(
#     temperature=0.5,
#     do_sample=True,
#     max_length=256,
#     forced_bos_token_id = 256011, # Arabic

#     pad_token_id=tokenizer.pad_token_id,
#     bos_token_id= 256011,
#     decoder_start_token_id= 2,
#     eos_token_id= tokenizer.eos_token_id,
    
# #     num_beams = 4,
# #     early_stopping=True,
# #     top_k=50,
    
# #     renormalize_logits=True,
    
# #     # Testing Config
# #       repetition_penalty=0.5,
# #     num_return_sequences=4, # Number of sentences to generate
# #     return_dict_in_generate=True, # Returns the complete generation data from within the model.
# #     output_scores=True, # Score of each token.
# )

# tokenizer.src_lang="eng_Latn"
# tokenizer.tgt_lang="arb_Arab"

# model.push_to_hub("Abdulmohsena/Faseeh_LoRA")
# tokenizer.push_to_hub("Abdulmohsena/Faseeh_LoRA")
# generation_config.push_to_hub("Abdulmohsena/Faseeh_LoRA")

In [7]:
# Instantiating The Model
model_name= "facebook/nllb-200-distilled-600M"
# model_name = "Abdulmohsena/faseeh_alter"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn", tgt_lang="arb_Arab")
# generation_config = GenerationConfig.from_pretrained(model_name)

generation_config = GenerationConfig(
    temperature=0.5,
    do_sample=True,
    max_length=256,
    forced_bos_token_id = 256011, # Arabic

    pad_token_id=tokenizer.pad_token_id,
    bos_token_id= 256011,
    decoder_start_token_id= 2,
    eos_token_id= tokenizer.eos_token_id,
    
#     num_beams = 4,
#     early_stopping=True,
#     top_k=50,
    
#     renormalize_logits=True,
    
#     # Testing Config
#       repetition_penalty=0.5,
#     num_return_sequences=4, # Number of sentences to generate
#     return_dict_in_generate=True, # Returns the complete generation data from within the model.
#     output_scores=True, # Score of each token.
)

tokenizer.src_lang="eng_Latn"
tokenizer.tgt_lang="arb_Arab"

# https://huggingface.co/docs/transformers/en/main_classes/text_generation

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [8]:
# # Compressing
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training, LoraRuntimeConfig
# from torch.profiler import profile, record_function, ProfilerActivity

# ## Quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=bnb_config)

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model) # prepares the whole model for kbit training

# for param in model.parameters():
#     param.requires_grad = False  # freeze the model - train adapters later
#     if param.ndim == 1:
#         # cast the small parameters (e.g. layernorm) to fp32 for stability
#         param.data = param.data.to(torch.float32)
    
# ## Low Rank Adaptation
# lora_config = LoraConfig(
# #     init_lora_weights="olora",
#     use_dora=True,
#     runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=True),
#     task_type=TaskType.SEQ_2_SEQ_LM,
#     inference_mode=False, 
#     r=16, 
#     lora_alpha=16, 
#     lora_dropout=0.05,
#     target_modules=["k_proj", "q_proj", "v_proj", "out_proj", "fc1", "fc2"],
# )


# # model.enable_input_require_grads()
# model = get_peft_model(model, lora_config)


# # # Only train decoder weights, not encoder
# for param in model.get_base_model().model.encoder.parameters():
#     param.requires_grad = False

# model.print_trainable_parameters()

# # Pruning, not valid because we need a sparse util
# # for name, module in model.named_modules():
# #     if isinstance(module, (torch.nn.Linear, torch.nn.Embedding)):
# #         prune.l1_unstructured(module, name='weight', amount=0.4)
# #         prune.remove(module, 'weight')

# # # https://huggingface.co/docs/optimum/en/concept_guides/quantization
# # # https://huggingface.co/docs/peft/en/index
# # # https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py

In [9]:
# Sanity Check
dummy = "And the Egyptian Foreign Minister assured the visitors that security is always a top priority."

model = model.to('cuda')
encoded_ar = tokenizer(dummy, return_tensors="pt").to('cuda')
generated_tokens = model.generate(**encoded_ar, generation_config=generation_config)

tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


'و أؤكد وزير الخارجية المصري للزوار أن الأمن هو دائما أولوية أساسية.'

In [10]:
dummy = "spreading rumors and jokes"

model = model.to('cuda')
encoded_ar = tokenizer(dummy, return_tensors="pt").to('cuda')
generated_tokens = model.generate(**encoded_ar, generation_config=generation_config)

tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

'يُنشر الشائعات والمزاح.'

In [11]:
general_ds = load_dataset("allenai/nllb", "arb_Arab-eng_Latn", split="train", streaming=True)
sample = general_ds.shuffle(seed=42, buffer_size=1000_000)

temp = list(sample.take(100_000))
temp = [t['translation'] for t in temp]
base_ds = Dataset.from_list(temp).rename_column("arb_Arab", "ar").rename_column("eng_Latn", "en")

README.md:   0%|          | 0.00/38.6k [00:00<?, ?B/s]

nllb.py:   0%|          | 0.00/9.49k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.05M [00:00<?, ?B/s]

nllb_lang_pairs.py:   0%|          | 0.00/81.9k [00:00<?, ?B/s]

The repository for allenai/nllb contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/nllb.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Repo card metadata block was not found. Setting CardData to empty.


In [12]:
import re

TASHKEEL_REGEX = re.compile(r"[\u064B-\u0652]")
BRACKETS_REGEX = re.compile(r"[\[\]()]")

def no_tashkeel_or_brackets(example):
    ar_text = example["ar"]
    return not (TASHKEEL_REGEX.search(ar_text) or BRACKETS_REGEX.search(ar_text))

filtered_dataset = base_ds.filter(no_tashkeel_or_brackets)

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [13]:
filtered_dataset[444]

{'ar': 'الا ان الميزانية التي اعلنت في 22 اذار مارس لم تذكر اي تغييرات في النظام القائم.',
 'en': 'But the budget, announced on March 22, makes no mention of changes to the existing regime.'}

In [14]:
from datasets import concatenate_datasets

dataset = load_dataset("Abdulmohsena/Classic-Arabic-English-Language-Pairs")

dataset = concatenate_datasets([
    dataset['quran'],
    dataset['hadith'],
    dataset['books']
])

dataset = dataset.shuffle(seed=42)

README.md:   0%|          | 0.00/952 [00:00<?, ?B/s]

quran-00000-of-00001.parquet:   0%|          | 0.00/818k [00:00<?, ?B/s]

hadith-00000-of-00001.parquet:   0%|          | 0.00/644k [00:00<?, ?B/s]

books-00000-of-00001.parquet:   0%|          | 0.00/3.00M [00:00<?, ?B/s]

Generating quran split:   0%|          | 0/9474 [00:00<?, ? examples/s]

Generating hadith split:   0%|          | 0/4107 [00:00<?, ? examples/s]

Generating books split:   0%|          | 0/13331 [00:00<?, ? examples/s]

In [15]:
complete_dataset = concatenate_datasets([filtered_dataset, dataset]).shuffle(seed=42)

In [16]:
print(f"Base dataset for stability: {filtered_dataset.num_rows / (complete_dataset.num_rows):.2%}")
print(f"Styled dataset for style and quality: {dataset.num_rows / (complete_dataset.num_rows):.2%}")

ratio_str = f"{(filtered_dataset.num_rows / complete_dataset.num_rows) * 100:.0f}_{(dataset.num_rows / complete_dataset.num_rows) * 100:.0f}"

Base dataset for stability: 73.40%
Styled dataset for style and quality: 26.60%


In [17]:
complete_dataset[111]

{'ar': 'في عام 1492، كان عدد قليل منهم قد نظر إلى مدن العالم، ويعتقد أن أوروبا سوف تأتي للسيطرة على قرون التجارة العالمية في وقت لاحق.',
 'en': 'In 1492, few would have looked at the cities of the world and believed that Europe would come to dominate global trade centuries later.'}

In [18]:
preprocess_function = lambda examples: tokenizer(
        examples['en'], text_target=examples['ar'], max_length=256, truncation=True, padding=True, return_tensors='pt')

tokenized_dataset = complete_dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.20, seed=42)

Map:   0%|          | 0/101159 [00:00<?, ? examples/s]

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True, return_tensors='pt')

## Training

### Reward model

In [20]:
from evaluate import load
sacrebleu = load("sacrebleu")

def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        bleu_score = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)['score']
        #fluency_score = fluency.compute(texts=decoded_preds)['classical_score']
        
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

        result = {
                    "bleu": bleu_score,
                    #"fluency": fluency_score,
                    "gen_len": np.mean(prediction_lens)
                }
        
        result = {k: round(v, 4) for k, v in result.items()}
        return result

## TEST

In [21]:
# Freeze until convergence then unfreeze

for param in model.get_encoder().parameters():
    param.requires_grad = False

In [22]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datetime import datetime

training_args = Seq2SeqTrainingArguments(
        output_dir="faseeh_alter",
        eval_strategy="epoch",
        weight_decay=0.01,
        warmup_steps=1_000,
        learning_rate=5e-5,
        lr_scheduler_type="cosine",
        per_device_train_batch_size=6,
        per_device_eval_batch_size=6,
        save_total_limit=2,
        num_train_epochs=5,
        predict_with_generate=True,
        seed = 42,
        report_to="wandb",
        run_name=f'{datetime.now()}',
        logging_strategy = 'steps',
        logging_steps = 300,
        # load_best_model_at_end = True,



        # fp16=True,
        push_to_hub=True,
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'] ,
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_49/2939897815.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
artifact = wandb.Artifact(
    name="Faseeh_thaw_" + ratio_str,
    type="model"
)

# Add the directory (where trainer saved the model)
artifact.add_dir(training_args.output_dir)  # or training_args.output_dir

# Log the artifact
wandb.log_artifact(artifact)
wandb.finish()

In [ ]:
# import torch.profiler as profiler
# with profiler.profile(activities=[profiler.ProfilerActivity.CPU, profiler.ProfilerActivity.CUDA]) as prof:
    

# print(prof.key_averages().table(sort_by="cuda_time_total"))